In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json
from datetime import datetime,timedelta
env = "production"
# env = "staging"
# 
s3_mobile_client_bucket = "s3://mist-secorapp-{env}/mobile-client-marvis/mobile-client-marvis-{env}/".format(env=env)

now = datetime.now()  - timedelta(hours=3)
date_day = now.strftime("%Y-%m-%d")
date_hour = now.strftime("%H")


date_day = "2020-07-0*"
date_hour = "*"

s3_mobile_client_path = s3_mobile_client_bucket + "dt={date}/{hr}/*.seq".format(date=date_day, hr=date_hour)
print(s3_mobile_client_path)



s3_client_stats_bucket = "s3://mist-secorapp-{env}/client-stats-analytics/client-stats-analytics-{env}/".format(env=env)


# date_day= datetime.datetime.today().strftime("%Y-%m-%d")
# hr = '*'

# date_day = "2020-05-27*"
# hr = '20'

s3_client_stats_path = s3_client_stats_bucket + "dt={date}/hr={hr}/*.parquet".format(date=date_day, hr=date_hour)

print(s3_client_stats_path)


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
26,application_1591015534858_0030,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
s3://mist-secorapp-production/mobile-client-marvis/mobile-client-marvis-production/dt=2020-07-0*/*/*.seq
s3://mist-secorapp-production/client-stats-analytics/client-stats-analytics-production/dt=2020-07-0*/hr=*/*.parquet

In [2]:

rdd_mobile_client = spark.sparkContext.sequenceFile(s3_mobile_client_path)


In [3]:
import pprint 
test= rdd_mobile_client.first()

test1 = json.loads(test[1])
pprint.pprint(test1)


{'HashKey': '794ddc71-bce7-496d-a0db-0615026f24c6',
 'InfoFromTerminator': {'ID': '9777c1a0-6ef6-11e6-8bbf-02e208b2d34f.794ddc71bce7496da0db0615026f24c6',
                        'OrgID': '9777c1a0-6ef6-11e6-8bbf-02e208b2d34f',
                        'RemoteAddr': '71.204.164.162:33960',
                        'SiteID': None,
                        'Terminator': 'client-terminator-001-production',
                        'Timestamp': '2020-07-01T00:00:03.281701668Z'},
 'MarvisDataTimestamp': '2020-07-01T00:00:06.3050Z',
 'ReplyPath': {'Opaque': 'AAAQy1uRZk7DAVLFpC9nHDMemJr8FISoBKCd5VYI56OdmvcJzypcGmxqjsSXJgKYSmgy9uKGUk6J62/xS/OASlsVX8JiNQLHra4CZ9wGgwQ7XnyNtvFIsHt4EKGm+SM8b4C1fokwh9XUpywH21uO24pVD37rJvXnuiwdf6xGQQHsnDKRSUnyixhl6lUQeMXKer+sy63y3tZ6vnQBgKGzirpT9wro0gazC3pm1CC61QF8Og==',
               'Partition': 9,
               'Topic': 'client-reply-client-terminator-001-production'},
 'Result': {'ClientInformation': {'ClientInfoTimestamp': '2020-07-01T00:00:06.2990Z',
           

In [4]:
len(test1["Result"]["Stats"]["Passive"]["Tests"])

29

In [13]:
import pprint
pprint.pprint(test1["Result"]["Stats"]["Passive"]["Tests"][0]["WiFiScanData"])

[]

In [14]:
df_mobile_client= rdd_mobile_client.map(lambda x: json.loads(x[1])).toDF()

# df_t = df_mobile_client.select("HashKey", col("Result.ClientInformation.Device.MAC").alias("MAC"),
#                                col("Result.Stats.Passive.Tests").alias("Tests"))
df_t = df_mobile_client.select( col("Result.ClientInformation.Device.MAC").alias("MAC")).groupBy("MAC").count().show()
#                                col("Result.Stats.Passive.Tests").alias("Tests"))

+-----------------+-----+
|              MAC|count|
+-----------------+-----+
|d4-63-c6-eb-1f-a2| 3345|
|08-c5-e1-aa-df-c7| 7744|
|b4-cd-27-83-36-f4|  183|
|94-fb-29-a6-7c-48|16924|
|ec-9b-f3-f1-20-01| 2123|
|     NotAvailable|10197|
|80-7a-bf-42-0b-93|  732|
|04-ba-8d-ba-0a-a2|  491|
|ec-1f-72-50-1e-5b|   72|
|08-c5-e1-1c-92-5e|  868|
+-----------------+-----+

In [7]:
rdd_t = rdd_mobile_client.map(lambda x: json.loads(x[1]))\
        .map(lambda x: {"MAC": x["Result"]["ClientInformation"]["Device"]["MAC"],
                        "Timestamp":x["Result"]["Stats"]["Passive"]["PassiveTimestamp"],
                        "Tests": x["Result"]["Stats"]["Passive"]["Tests"]
                       }
        )

# df_t = rdd_t.toDF()
# df_t.printSchema()

In [8]:
rdd_t.first()

{'MAC': '94-fb-29-a6-7c-48', 'Timestamp': '2020-07-01T00:00:06.3000Z', 'Tests': [{'ConnectionType': 'WiFi', 'CellularData': {}, 'WiFiConnectionData': {'BSSID': '5c:5b:35:31:5c:f3', 'SignalStrength': 99, 'LinkSpeed': 144, 'Frequency': 2437, 'RSSI': -51, 'SSID': 'LIVE_DEMO', 'NetworkId': 11, 'SupplicantState': 'COMPLETED', 'IP': '192.168.0.27'}, 'WiFiScanData': [], 'BackgroundMode': 'true', 'AirplaneModeOn': 'false', 'Timestamp': '2020-06-30T23:59:37.0890Z'}, {'ConnectionType': 'WiFi', 'CellularData': {}, 'WiFiConnectionData': {'BSSID': '5c:5b:35:31:5c:f3', 'SignalStrength': 99, 'LinkSpeed': 144, 'Frequency': 2437, 'RSSI': -51, 'SSID': 'LIVE_DEMO', 'NetworkId': 11, 'SupplicantState': 'COMPLETED', 'IP': '192.168.0.27'}, 'WiFiScanData': [], 'BackgroundMode': 'true', 'AirplaneModeOn': 'false', 'Timestamp': '2020-06-30T23:59:38.1320Z'}, {'ConnectionType': 'WiFi', 'CellularData': {}, 'WiFiConnectionData': {'BSSID': '5c:5b:35:31:5c:f3', 'SignalStrength': 99, 'LinkSpeed': 144, 'Frequency': 2437

In [9]:
rdd_t = rdd_mobile_client.map(lambda x: json.loads(x[1]))\
        .map(lambda x: {"MAC": x["Result"]["ClientInformation"]["Device"]["MAC"],
                        "Timestamp":x["Result"]["Stats"]["Passive"]["PassiveTimestamp"],
                        "Tests": x["Result"]["Stats"]["Passive"]["Tests"]
                       }
        )

def flat_scan(x):
    res=[]
    mac = x.get("MAC")
#     ts = x.get("Timestamp")
    for test in x.get("Tests"):
        for scan in test.get("WiFiScanData"):
            scan["MAC"] = mac
            
            res.append(scan)
    return res

# rdd_t.flatMap(lambda x: [xx.get("WiFiScanData") for xx in x.get("Tests")]).first()

df_scan = rdd_t.flatMap(lambda x: flat_scan(x)).toDF()
df_scan.printSchema()

root
 |-- 80211mcResponder: boolean (nullable = true)
 |-- BSSID: string (nullable = true)
 |-- CenterFreq0: long (nullable = true)
 |-- CenterFreq1: long (nullable = true)
 |-- ChannelBandwidth: string (nullable = true)
 |-- Frequency: long (nullable = true)
 |-- Level: long (nullable = true)
 |-- MAC: string (nullable = true)
 |-- SSID: string (nullable = true)
 |-- WifiScanTimeStamp: string (nullable = true)

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead

In [10]:
df_scan.show(3)

+----------------+-----------------+-----------+-----------+----------------+---------+-----+-----------------+------------+--------------------+
|80211mcResponder|            BSSID|CenterFreq0|CenterFreq1|ChannelBandwidth|Frequency|Level|              MAC|        SSID|   WifiScanTimeStamp|
+----------------+-----------------+-----------+-----------+----------------+---------+-----+-----------------+------------+--------------------+
|           false|10:da:43:d2:f0:07|       5755|          0|              40|     5765|  -61|08-c5-e1-aa-df-c7|NETGEAR85-5G|2020-07-01T01:20:...|
|           false|10:da:43:d2:f0:07|       5755|          0|              40|     5765|  -61|08-c5-e1-aa-df-c7|NETGEAR85-5G|2020-07-01T01:20:...|
|           false|10:da:43:d2:f0:07|       5755|          0|              40|     5765|  -61|08-c5-e1-aa-df-c7|NETGEAR85-5G|2020-07-01T01:20:...|
+----------------+-----------------+-----------+-----------+----------------+---------+-----+-----------------+------------+

In [11]:
c1 = "ec:1f:72:50:1e:5b".replace(":", "-")
print(c1)
df_scan_c1 = df_scan.filter(col("MAC")==c1)
df_scan_c1.show()

ec-1f-72-50-1e-5b
+----------------+-----+-----------+-----------+----------------+---------+-----+---+----+-----------------+
|80211mcResponder|BSSID|CenterFreq0|CenterFreq1|ChannelBandwidth|Frequency|Level|MAC|SSID|WifiScanTimeStamp|
+----------------+-----+-----------+-----------+----------------+---------+-----+---+----+-----------------+
+----------------+-----+-----------+-----------+----------------+---------+-----+---+----+-----------------+

In [12]:
df_scan.select("MAC").groupBy("MAC").count().show()
# c1

+-----------------+------+
|              MAC| count|
+-----------------+------+
|d4-63-c6-eb-1f-a2| 95826|
|08-c5-e1-aa-df-c7|529276|
|b4-cd-27-83-36-f4| 10163|
|94-fb-29-a6-7c-48|161608|
|ec-9b-f3-f1-20-01| 89323|
|     NotAvailable|302618|
|80-7a-bf-42-0b-93| 37760|
|04-ba-8d-ba-0a-a2| 19310|
|08-c5-e1-1c-92-5e| 18600|
+-----------------+------+

In [16]:
c2 = "08-c5-e1-aa-df-c7"
df_scan_c2 = df_scan.filter(col("MAC")==c2)
df_scan_c2.show()

+----------------+-----------------+-----------+-----------+----------------+---------+-----+-----------------+------------+--------------------+
|80211mcResponder|            BSSID|CenterFreq0|CenterFreq1|ChannelBandwidth|Frequency|Level|              MAC|        SSID|   WifiScanTimeStamp|
+----------------+-----------------+-----------+-----------+----------------+---------+-----+-----------------+------------+--------------------+
|           false|10:da:43:d2:f0:07|       5755|          0|              40|     5765|  -61|08-c5-e1-aa-df-c7|NETGEAR85-5G|2020-07-01T01:20:...|
|           false|10:da:43:d2:f0:07|       5755|          0|              40|     5765|  -61|08-c5-e1-aa-df-c7|NETGEAR85-5G|2020-07-01T01:20:...|
|           false|10:da:43:d2:f0:07|       5755|          0|              40|     5765|  -61|08-c5-e1-aa-df-c7|NETGEAR85-5G|2020-07-01T01:20:...|
|           false|10:da:43:d2:f0:07|       5755|          0|              40|     5765|  -61|08-c5-e1-aa-df-c7|NETGEAR85-5G|

In [38]:
import pandas as pd

In [39]:
df= df_scan_c2.select("*").toPandas()
df.head()

   80211mcResponder              BSSID  ...          SSID          WifiScanTimeStamp
0             False  10:da:43:d2:f0:07  ...  NETGEAR85-5G  2020-07-01T01:20:31.3469Z
1             False  10:da:43:d2:f0:07  ...  NETGEAR85-5G  2020-07-01T01:20:32.3720Z
2             False  10:da:43:d2:f0:07  ...  NETGEAR85-5G  2020-07-01T01:20:33.3540Z
3             False  10:da:43:d2:f0:07  ...  NETGEAR85-5G  2020-07-01T01:20:34.3370Z
4             False  10:da:43:d2:f0:07  ...  NETGEAR85-5G  2020-07-01T01:20:35.3140Z

[5 rows x 10 columns]

In [40]:
df.to_csv("df_scan.csv")


[Errno 13] Permission denied: 'df_scan.csv'
Traceback (most recent call last):
  File "/usr/lib64/python3.6/dist-packages/pandas/core/generic.py", line 3020, in to_csv
    formatter.save()
  File "/usr/lib64/python3.6/dist-packages/pandas/io/formats/csvs.py", line 157, in save
    compression=self.compression)
  File "/usr/lib64/python3.6/dist-packages/pandas/io/common.py", line 424, in _get_handle
    f = open(path_or_buf, mode, encoding=encoding, newline="")
PermissionError: [Errno 13] Permission denied: 'df_scan.csv'



In [35]:
# import matplotlib. as plt

import matplotlib.pyplot as plt
df.plot(x="Level")


No module named 'matplotlib'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'matplotlib'



In [37]:
%%!
pip install matplotlib

['Requirement already satisfied: matplotlib in /opt/conda/lib/python3.6/site-packages (3.1.1)',
 'Requirement already satisfied: cycler>=0.10 in /opt/conda/lib/python3.6/site-packages (from matplotlib) (0.10.0)',
 'Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /opt/conda/lib/python3.6/site-packages (from matplotlib) (2.4.2)',
 'Requirement already satisfied: numpy>=1.11 in /opt/conda/lib/python3.6/site-packages (from matplotlib) (1.18.1)',
 'Requirement already satisfied: python-dateutil>=2.1 in /opt/conda/lib/python3.6/site-packages (from matplotlib) (2.8.0)',
 'Requirement already satisfied: kiwisolver>=1.0.1 in /opt/conda/lib/python3.6/site-packages (from matplotlib) (1.1.0)',
 'Requirement already satisfied: six in /opt/conda/lib/python3.6/site-packages (from cycler>=0.10->matplotlib) (1.14.0)',
 'Requirement already satisfied: setuptools in /opt/conda/lib/python3.6/site-packages (from kiwisolver>=1.0.1->matplotlib) (41.2.0)']

In [ ]:
for test in test1["Result"]["Stats"]["Passive"]["Tests"]:
    print(len(test.get("WiFiScanData")))

In [ ]:
df_t.show(3)

In [ ]:
df_t.select("MAC","Timestamp",  explode("Tests")).show(1, False)

In [ ]:
df_t1 =df_t.select("MAC", "Timestamp",  explode(col("Tests")).alias("Tests"))

df_t1.show()

In [ ]:
df_t1.printSchema()

In [ ]:
df_t1.select("Tests")

In [ ]:
def get_wifiscan(x):
    return x[0]["WiFiScanData"]

df_t1.withColumn("WifiScan", get_wifiscan("Tests")).show()

In [ ]:
df_t.printSchema()

In [ ]:
df_t.show()

In [ ]:
df_t.select("MAC", explode("Tests")).show()